In [22]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;


In [27]:
#name_plan='2D_SIN_PTV_5_35'
#list_suffix=['A5T5', 'A10T5', 'A15T5', 'A20T5', 'A25T2', 'A25T3', 'A25T7']
#list_suffix=['A5T5', 'A10T5', 'A15T5', 'A20T5', 'A25T2', 'A25T3', 'A25T7', 'A25T10']
#list_suffix=['AP_A5T5', 'AP_A10T5', 'AP_A15T5', 'SI_A5T5', 'SI_A10T5', 'SI_A15T5', 'SI_A20T5']
#list_suffix=['AP_A5T5', 'AP_A10T5', 'AP_A15T5', 'SI_A5T5', 'SI_A10T5', 'SI_A15T5', 'SI_A20T5', 'LR_A5T5', 'LR_A7T5', 'LR_A10T5', 'LR_A12T5']
#list_plan=['1D_SIN_ITV_10', '1D_SIN_ITV_15', '1D_SIN_ITV_25', '1D_SIN_ITV_30','2D_SIN_ITV_5_15', '2D_SIN_ITV_5_35','2D_SIN_ITV_10_15', '2D_SIN_ITV_5_25','2D_SIN_PTV_5_5','2D_SIN_PTV_5_15','2D_SIN_PTV_5_35']  
#list_plan=['2D_CUR_ITV_5_5' , '2D_CUR_ITV_5_15', '2D_CUR_ITV_5_25', '2D_CUR_ITV_5_35','2D_CUR_ITV_10_5', '2D_CUR_ITV_10_15','2D_CUR_ITV_10_25', '2D_CUR_ITV_10_35']
list_plan=['3D_SIN_ITV_5_5_7' , '3D_SIN_ITV_5_15_7', '3D_SIN_ITV_5_25_7', '3D_SIN_ITV_5_35_7',  '3D_SIN_ITV_10_5_7', '3D_SIN_ITV_10_15_7','3D_SIN_ITV_10_25_7', '3D_SIN_ITV_10_35_7']#,'2D_SIN_PTV_5_5','2D_SIN_PTV_5_15','2D_SIN_PTV_5_35']  
list_plan=['3D_SIN_ITV_5_35_7']#,  '3D_SIN_ITV_10_5_7', '3D_SIN_ITV_10_15_7','3D_SIN_ITV_10_25_7', '3D_SIN_ITV_10_35_7']#,'2D_SIN_PTV_5_5','2D_SIN_PTV_5_15','2D_SIN_PTV_5_35']  
list_simu=['_dose_no_interplay' ]
list_suffix=['_T5']#, '_static']
#rp=dcm.read_file('../interplay/output/new/RTplan/RP_'+name_plan+'.dcm')

In [28]:
for plan in list_plan:
    for simu in list_simu:
            for suf in list_suffix:
                rp=dcm.read_file('../interplay/output/new/RTplan/RP_'+plan+'.dcm')
                name='interplay_'+plan+suf+simu
                r='/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/results/'
                gateim=sitk.ReadImage(r+name+'/'+name+'.mhd')
                # get pixels arrays of images and vizualization
                gatepix=sitk.GetArrayFromImage(gateim)
                print(name)
                #ct_viewer(gatepix)
                # compute true value gate output offset in the DICOM frame
                new_origin=np.zeros(3)
                #print(gateim.GetOrigin())
                new_origin[0]=gateim.GetOrigin()[0]
                new_origin[1]=gateim.GetOrigin()[1]*-1
                new_origin[2]=gateim.GetOrigin()[2]*-1
                gateim.SetOrigin(new_origin)
                #print('New Origin : ', gateim.GetOrigin())
                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                #FACTEUR A TROUVER POUR LES PHOTONS !!!!!!!!!!!!!!!!
                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                # ************ F for interplay ************
                E=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
                mode=rp.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode
                energy=str(E)+mode
                #energy="6STANDARD"
                #print(mode)
                if energy == "6NON_STANDARD":
                    Dose_Gate_ref=0.0005063408
                    N_part_ref=130170012500.0


                if energy == "6STANDARD":          #avec les killboxes
                    Dose_Gate_ref=0.00033116873      #0.00033096725  
                    N_part_ref=127666770000.0

                #nb de part pour faire le phsp réduit
                N_part_simu=2606532517
                #print(energy)
                #Absolute dose conversion
                um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
                #fraction unique for interplay
                nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

                #print('um:', um)
                #print('frac:', nFrac)

                mask=np.ones_like(gatepix)
                gatepix_mask=gatepix*mask

                gatepix_abs=(gatepix_mask/N_part_simu)*(0.01)*(N_part_ref/Dose_Gate_ref)*um/100 #*nFrac #un seule fraction here
                #gatepix_abs=gatepix_abs/3.55 #scaling factor no interplay F = nb phantom position/100
                #print("max avant calib:", gatepix_mask.max())
                print('maximum:', gatepix_abs.max())

                #Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
                gate_dose=sitk.GetImageFromArray(gatepix_abs)
                gate_dose.SetOrigin(gateim.GetOrigin())
                gate_dose.SetSpacing(gateim.GetSpacing())
                sitk.WriteImage(gate_dose, r+name+'/'+name+'_abs.mhd')
                print('')


interplay_3D_SIN_ITV_5_35_7_T5_dose_no_interplay
maximum: 2.043861580486896



In [5]:
for suf in list_suffix:
    name='interplay_'+name_plan+suf+'_dose_no_interplay'
    r='/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/results/'
    gateim=sitk.ReadImage(r+name+'/'+name+'.mhd')
    # get pixels arrays of images and vizualization
    gatepix=sitk.GetArrayFromImage(gateim)
    print(name)
    #ct_viewer(gatepix)
    # compute true value gate output offset in the DICOM frame
    new_origin=np.zeros(3)
    #print(gateim.GetOrigin())
    new_origin[0]=gateim.GetOrigin()[0]
    new_origin[1]=gateim.GetOrigin()[1]*-1
    new_origin[2]=gateim.GetOrigin()[2]*-1
    gateim.SetOrigin(new_origin)
    #print('New Origin : ', gateim.GetOrigin())
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #FACTEUR A TROUVER POUR LES PHOTONS !!!!!!!!!!!!!!!!
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # ************ F for interplay ************
    E=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
    mode=rp.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode
    energy=str(E)+mode
    #energy="6STANDARD"
    #print(mode)
    if energy == "6NON_STANDARD":
        Dose_Gate_ref=0.0005063408
        N_part_ref=130170012500.0


    if energy == "6STANDARD":          #avec les killboxes
        Dose_Gate_ref=0.00033116873      #0.00033096725  
        N_part_ref=127666770000.0

    #nb de part pour faire le phsp réduit
    N_part_simu=2606532517
    #print(energy)
    #Absolute dose conversion
    um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
    #fraction unique for interplay
    nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

    #print('um:', um)
    #print('frac:', nFrac)

    mask=np.ones_like(gatepix)
    gatepix_mask=gatepix*mask

    gatepix_abs=(gatepix_mask/N_part_simu)*(0.01)*(N_part_ref/Dose_Gate_ref)*um/100 #*nFrac #un seule fraction here
    #gatepix_abs=gatepix_abs/3.55 #scaling factor no interplay F = nb phantom position/100
    #print("max avant calib:", gatepix_mask.max())
    print('maximum:', gatepix_abs.max())

    #Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
    gate_dose=sitk.GetImageFromArray(gatepix_abs)
    gate_dose.SetOrigin(gateim.GetOrigin())
    gate_dose.SetSpacing(gateim.GetSpacing())
    sitk.WriteImage(gate_dose, r+name+'/'+name+'_abs.mhd')
    print('')
    

interplay_2D_SIN_PTV_5_35_T3_dose_no_interplay
maximum: 2.05343821039519

interplay_2D_SIN_PTV_5_35_T5_dose_no_interplay
maximum: 2.0439050055388046

interplay_2D_SIN_PTV_5_35_T7_dose_no_interplay
maximum: 2.045923274918895

